'''for index, i in data.iterrows():
    
    room=i['Cabin']
    print(index, room)
    if not pd.isnull(room):  
        data.at[index, 'Deck']=room[0]
data=data.drop(['Cabin'], axis=1)'''

PRE-PROCESSING

In [12]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv('Rents.csv', sep=',', encoding='iso8859_4')
print(data.head(5))
print(data.shape)
nums=["0"]*58548
data['Posti']=nums
data['Kaupunginosa']=nums
data['Kaupunki']=nums

# creates new rows for postcode, city and district
for index, row in data.iterrows():
    full=row['Postinumero']
    post=full[0:6]
    full=full[6:]
    i=0
    for chr in full:
        if chr=='(':
            paikka=full[0:i]
            kaupunki=full[i+1:-1]
            break
        i+=1   
    post = re.sub(r'\s', '', post)
    kaupunki = re.sub(r'\s', '', kaupunki)
    paikka = re.sub(r'\s', '', paikka)

    # add nans for missing points
    if row['Neliövuokra (eur/m2)'] == ".": 
        data.at[index, 'Neliövuokra (eur/m2)']= np.nan
    else:
        data.at[index, 'Neliövuokra (eur/m2)']= float(row['Neliövuokra (eur/m2)'])
    
    data.loc[index, 'Posti'] = post
    data.loc[index, 'Kaupunginosa'] = paikka
    data.loc[index, 'Kaupunki'] = kaupunki
    #print(post, paikka, kaupunki, "je")
data=data.drop(['Postinumero'], axis=1)

# important cities
remainers = ['Helsinki', 'Vantaa', 'Espoo', 'Turku', 'Tampere']
data=data.where(data['Kaupunki'].isin(remainers)).dropna(subset=['Kaupunki']) 
data['Vuosineljännes']=data['Vuosineljännes'].astype(str)
data['Neliövuokra (eur/m2)']=data['Neliövuokra (eur/m2)'].astype(float)
# data['Neliövuokra (eur/m2)']=data['Neliövuokra (eur/m2)'].astype(float) #not needed anymore?
print(data.head(3))


  Vuosineljännes                                        Postinumero Huoneluku  \
0         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )    Yksiöt   
1         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )   Kaksiot   
2         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )  Kolmiot+   
3         2015Q1                      00120 Punavuori   (Helsinki )    Yksiöt   
4         2015Q1                      00120 Punavuori   (Helsinki )   Kaksiot   

  Lukumäärä Neliövuokra (eur/m2)  
0       452                24.26  
1       243                18.36  
2        34                13.09  
3        95                21.52  
4        52                14.77  
(58548, 5)
  Vuosineljännes Huoneluku Lukumäärä  Neliövuokra (eur/m2)  Posti  \
0         2015Q1    Yksiöt       452                 24.26  00100   
1         2015Q1   Kaksiot       243                 18.36  00100   
2         2015Q1  Kolmiot+        34                 13.09  00100   

               

In [37]:
#old dont run

huoneet=['Yksiöt', 'Kaksiot', 'Kolmiot+']
setit=set()
uutuus = pd.DataFrame().reindex_like(data)
#print(uutuus.head())
i =0

for index, row in data.iterrows():
    post=row['Posti']
    #print(post)
    if post not in setit:
        setit.add(post)
        for huone in huoneet:
            pala=data.loc[(data['Huoneluku']==huone)& (data['Posti']==post)]
            #print(pala.head(5))
            pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
            #print(pala.head(5))
            for ind, row in pala.iterrows():
                uutuus.loc[i]=row
                i+=1
uutuus=uutuus.dropna(subset=['Kaupunki'])                

C:\Users\tarmo\AppData\Local\Temp\ipykernel_17108\1291929597.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
C:\Users\tarmo\AppData\Local\Temp\ipykernel_17108\1291929597.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015Q1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_17108\1291929597.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yksiöt' has dtype incompatible with float64, please explicitly cast to a com

KeyboardInterrupt: 

In [3]:
#READ INFINDEX FOR FILL

import numpy as np
import pandas as pd

infindex=pd.read_csv('001_11xb_2023m08_20231004-151722.csv', sep=',', encoding='iso8859_4')
vuokraindeksit = infindex.iloc[::2].reset_index(drop=True)
asuntoindeksit = infindex.iloc[1::2].reset_index(drop=True)
rent_inds=pd.DataFrame(columns=['Vuosineljännes', 'Index'])
house_inds=pd.DataFrame(columns=['Vuosineljännes', 'Index'])

i=0
while i<2:
    if i==0:
       to_be=vuokraindeksit
    else:
       to_be=asuntoindeksit
    i+=1
    summa=0
    year=2015
    quart=1
    counter=0
    for ind, row in to_be.iterrows():
        summa+=row['Indeksipisteluku']
        counter+=1
        if counter==3:
            counter=0
            summa/=3
            if i==1:
                time=f"{year}Q{quart}"
                new_row={'Vuosineljännes': time,'Index': summa}
                rent_inds.loc[len(rent_inds)] = new_row
            else:
                time=f"{year}Q{quart}"
                new_row={'Vuosineljännes': time,'Index': summa}
                house_inds.loc[len(house_inds)] = new_row
            quart+=1
            summa=0
            if quart==5:
                quart=1
                year+=1            


In [13]:
#FILL NAN WITH INDEX (WORKING)

import numpy as np
import pandas as pd

#infindex=pd.read_csv('001_11xb_2023m08_20231004-151722.csv', sep=',', encoding='iso8859_4')

huoneet=['Yksiöt', 'Kaksiot', 'Kolmiot+']
setit=set()
uutuus = pd.DataFrame().reindex_like(data)
#print(uutuus.head())
i =0
for index, row in data.iterrows():
    post=row['Posti']
    #print(post)
    if post not in setit:
        setit.add(post)
        for huone in huoneet:
            pala=data.loc[(data['Huoneluku']==huone) & (data['Posti']==post)]
            #print(pala.head(5))
            nanit=[]
            vall=0
            for ind, quarter in pala.iterrows():
                #print("quarter", quarter)
                temp=quarter['Neliövuokra (eur/m2)']
                #print(pd.isna(temp))
                #print(temp)
                if np.isnan(quarter['Neliövuokra (eur/m2)']):
                    #print("added")
                    nanit.append(quarter['Vuosineljännes'])
                else:
                    vall=quarter['Neliövuokra (eur/m2)']
                    quart=quarter['Vuosineljännes']
                    #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==quart)].values[0]
                    #inde=inde['Index']/100
                    inde = rent_inds.loc[rent_inds['Vuosineljännes'] == quart, 'Index'].values[0] / 100
                    vall=round(vall/inde, 2)
                    for nani in nanit:
                        #print("nani ", nani)
                        #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==nani)].values[0]
                        #inde=inde['Index']/100
                        inde = rent_inds.loc[rent_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == nani, 'Neliövuokra (eur/m2)'] = round(vall * inde, 2)
                        print(pala.loc[pala['Vuosineljännes'] == nani, 'Neliövuokra (eur/m2)'])
                    nanit=[]
            #print("through 1")
            if vall!=0:
                if len(nanit)>0:
                    for nani in nanit:
                        #print("ff", nani)
                        #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==nani)].values[0]
                        #inde=inde['Index']/100
                        inde = rent_inds.loc[rent_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == str(nani), 'Neliövuokra (eur/m2)'] = round(vall * inde, 2)
                        #print(pala.loc[pala['Vuosineljännes'] == str(nani), 'Neliövuokra (eur/m2)'], round(vall * inde, 2))
            nanit=[]
                #pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
            #print(pala.head(5))
            #print("through")
            for ind, row in pala.iterrows():
                uutuus.loc[i]=row
                i+=1
uutuus=uutuus.dropna(subset=['Kaupunki'])  

53384    23.07
Name: Neliövuokra (eur/m2), dtype: float64
5    16.34
Name: Neliövuokra (eur/m2), dtype: float64
1727    16.5
Name: Neliövuokra (eur/m2), dtype: float64
3449    16.6
Name: Neliövuokra (eur/m2), dtype: float64
5171    16.67
Name: Neliövuokra (eur/m2), dtype: float64
6893    16.75
Name: Neliövuokra (eur/m2), dtype: float64
8615    16.88
Name: Neliövuokra (eur/m2), dtype: float64
10337    16.97
Name: Neliövuokra (eur/m2), dtype: float64
12059    17.03
Name: Neliövuokra (eur/m2), dtype: float64
13781    17.11
Name: Neliövuokra (eur/m2), dtype: float64
15503    17.23
Name: Neliövuokra (eur/m2), dtype: float64
17225    17.31
Name: Neliövuokra (eur/m2), dtype: float64
18947    17.37
Name: Neliövuokra (eur/m2), dtype: float64
20669    17.44
Name: Neliövuokra (eur/m2), dtype: float64
22391    17.55
Name: Neliövuokra (eur/m2), dtype: float64
24113    17.63
Name: Neliövuokra (eur/m2), dtype: float64
25835    17.69
Name: Neliövuokra (eur/m2), dtype: float64
27557    17.78
Name: Neli

C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\1804632005.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015Q1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\1804632005.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yksiöt' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\1804632005.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '452' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\1804632005.py:61: FutureWarning: Setting an item

11    15.35
Name: Neliövuokra (eur/m2), dtype: float64
1733    15.5
Name: Neliövuokra (eur/m2), dtype: float64
3455    15.6
Name: Neliövuokra (eur/m2), dtype: float64
5177    15.66
Name: Neliövuokra (eur/m2), dtype: float64
6899    15.74
Name: Neliövuokra (eur/m2), dtype: float64
8621    15.86
Name: Neliövuokra (eur/m2), dtype: float64
10343    15.94
Name: Neliövuokra (eur/m2), dtype: float64
12065    16.0
Name: Neliövuokra (eur/m2), dtype: float64
13787    16.08
Name: Neliövuokra (eur/m2), dtype: float64
15509    16.19
Name: Neliövuokra (eur/m2), dtype: float64
17231    16.26
Name: Neliövuokra (eur/m2), dtype: float64
18953    16.32
Name: Neliövuokra (eur/m2), dtype: float64
20675    16.39
Name: Neliövuokra (eur/m2), dtype: float64
22397    16.49
Name: Neliövuokra (eur/m2), dtype: float64
24119    16.56
Name: Neliövuokra (eur/m2), dtype: float64
25841    16.62
Name: Neliövuokra (eur/m2), dtype: float64
27563    16.7
Name: Neliövuokra (eur/m2), dtype: float64
29285    16.82
Name: Neliö

In [3]:
import numpy as np
import pandas as pd

#NEW FILL THAT NOT WORK DONT RUN CRINGHE

huoneet = ['Yksiöt', 'Kaksiot', 'Kolmiot+']
setit = set()
uutuus = pd.DataFrame().reindex_like(data)
i = 0

for index, row in data.iterrows():
    post = row['Posti']
    
    if post not in setit:
        setit.add(post)
        
        for huone in huoneet:
            pala = data.loc[(data['Huoneluku'] == huone) & (data['Posti'] == post)]
            nanit = []
            vall = 0
            
            for ind, quarter in pala.iterrows():
                temp = quarter['Neliövuokra (eur/m2)']
                
                if temp == np.nan:
                    nanit.append(quarter['Vuosineljännes'])
                else:
                    vall = quarter['Neliövuokra (eur/m2)']
                    quart = quarter['Vuosineljännes']
                    inde = rent_inds.loc[rent_inds['Vuosineljännes'] == quart, 'Index'].values[0] / 100
                    vall = round(vall / inde, 2)
                    
                    for nani in nanit:
                        inde = rent_inds.loc[rent_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == nani, 'Neliövuokra (eur/m2)'] = round(vall * inde, 2)
                    
                    nanit = []
            
            if vall != 0:
                if len(nanit) > 0:
                    for nani in nanit:
                        inde = rent_inds.loc[rent_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == nani, 'Neliövuokra (eur/m2)'] = round(vall * inde, 2)
                
                nanit = []
                
            for ind, row in pala.iterrows():
                uutuus.loc[i] = row
                i += 1

uutuus = uutuus.dropna(subset=['Kaupunki'])

C:\Users\tarmo\AppData\Local\Temp\ipykernel_13740\1468406916.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015Q1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i] = row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_13740\1468406916.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yksiöt' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i] = row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_13740\1468406916.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '452' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i] = row
C:\Users\tarmo\AppData\Local\Temp\ipykernel_13740\1468406916.py:48: FutureWarning: Setti

ARIMA

# Prices 1/2/3


In [9]:
import pandas as pd
import numpy as np
import re

def Data_wrangle(data : pd.DataFrame, nan_row : str):
    # nan_row name of the row with . or ... to be replaced with np.nan
    # creates new rows for postcode, city and district
    nums=[0]*len(data)
    data['Posti']=nums
    data['Kaupunginosa']=nums
    data['Kaupunki']=nums
    for index, row in data.iterrows():
        full=row['Postinumero']
        post=full[0:6]
        full=full[6:]
        i=0
        for chr in full:
            if chr=='(':
                paikka=full[0:i]
                kaupunki=full[i+1:-1]
                break
            i+=1   
        post = re.sub(r'\s', '', post)
        kaupunki = re.sub(r'\s', '', kaupunki)
        paikka = re.sub(r'\s', '', paikka)
        
        # add nans for missing points
        if row[nan_row] == "." or row[nan_row] == "...": 
            data.at[index, nan_row]= np.nan
        else:
            data.at[index, nan_row]= float(row[nan_row])

        data.at[index, 'Posti']= post
        data.at[index, 'Kaupunginosa']= paikka
        data.at[index, 'Kaupunki']= kaupunki 
        #print(post, paikka, kaupunki, "je")
    data=data.drop(['Postinumero'], axis=1)

    # important cities
    remainers = ['Helsinki', 'Vantaa', 'Espoo', 'Turku', 'Tampere']
    data=data.where(data['Kaupunki'].isin(remainers)).dropna(subset=['Kaupunki']) 
    return data

data_pr = pd.concat([Data_wrangle(pd.read_csv(x, sep=',', encoding='iso8859_4'), 'Neliöhinta (EUR/m2)') for x in ['Prices 1bed.csv', 'Prices 2bed.csv', 'Prices 3bed.csv']]) #reads all 3 and wrangles them
data = Data_wrangle(pd.read_csv('Rents.csv', sep=',', encoding='iso8859_4'), 'Neliövuokra (eur/m2)')

data_pr.rename(columns={'Talotyyppi': 'Huoneluku', 'Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen' : 'Lukumäärä'}, inplace=True)

# values in Talotyyppi to same as the rent
to_rplc = {'Kerrostalo yksiöt': 'Yksiöt', 'Kerrostalo kaksiot': 'Kaksiot', 'Kerrostalo kolmiot+': 'Kolmiot+'}
data_pr['Huoneluku'].replace(to_rplc, inplace=True) 
print(data_pr.head(3))
data_pr.drop(data_pr.loc[(data_pr['Vuosineljännes']== '2023Q2*') | (data_pr['Vuosineljännes']== '2023Q1*')].index, inplace=True)
print(data.head(3))
drop_idx_1 = data_pr.loc[(data_pr['Huoneluku'] == 'Yksiöt') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
drop_idx_2 = data_pr.loc[(data_pr['Huoneluku'] == 'Kaksiot') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
drop_idx_3 = data_pr.loc[(data_pr['Huoneluku'] == 'Kolmiot+') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
data_pr = pd.concat([data_pr.loc[data_pr['Huoneluku']=='Yksiöt'][data_pr.loc[data_pr['Huoneluku']=='Yksiöt'].index > drop_idx_1], 
                     data_pr.loc[data_pr['Huoneluku']=='Kaksiot'][data_pr.loc[data_pr['Huoneluku']=='Kaksiot'].index > drop_idx_2],
                     data_pr.loc[data_pr['Huoneluku']=='Kolmiot+'][data_pr.loc[data_pr['Huoneluku']=='Kolmiot+'].index > drop_idx_3]
                     ])

C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\3678009960.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00100' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Posti']= post
C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\3678009960.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Helsinkikeskusta-Etu-Töölö' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupunginosa']= paikka
C:\Users\tarmo\AppData\Local\Temp\ipykernel_7784\3678009960.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Helsinki' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupunki']= kaupunki
C:\Users\tarmo\AppData\Loca

  Vuosineljännes Huoneluku Neliöhinta (EUR/m2) Lukumäärä  Posti  \
0         2009Q1    Yksiöt              4751.0        36  00100   
1         2009Q1    Yksiöt              4561.0         8  00120   
2         2009Q1    Yksiöt                 NaN         1  00130   

                 Kaupunginosa  Kaupunki  
0  Helsinkikeskusta-Etu-Töölö  Helsinki  
1         Punavuori-Bulevardi  Helsinki  
2             Kaartinkaupunki  Helsinki  
  Vuosineljännes Huoneluku Lukumäärä Neliövuokra (eur/m2)  Posti  \
0         2015Q1    Yksiöt       452                24.26  00100   
1         2015Q1   Kaksiot       243                18.36  00100   
2         2015Q1  Kolmiot+        34                13.09  00100   

                 Kaupunginosa  Kaupunki  
0  HelsinkiKeskusta-Etu-Töölö  Helsinki  
1  HelsinkiKeskusta-Etu-Töölö  Helsinki  
2  HelsinkiKeskusta-Etu-Töölö  Helsinki  


In [8]:
# data_pr = data_prices
# Change Talotyyppi -> Huoneluku, Kauppojen lukumäärä, vara... -> Lukumäärä
data_pr.rename(columns={'Talotyyppi': 'Huoneluku', 'Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen' : 'Lukumäärä'}, inplace=True)

# values in Talotyyppi to same as the rent
to_rplc = {'Kerrostalo yksiöt': 'Yksiöt', 'Kerrostalo kaksiot': 'Kaksiot', 'Kerrostalo kolmiot+': 'Kolmiot+'}
data_pr['Huoneluku'].replace(to_rplc, inplace=True) 

# drop 2023Q2*/Q1* from both
data_pr.drop(data_pr.loc[(data_pr['Vuosineljännes']== '2023Q2*') | (data_pr['Vuosineljännes']== '2023Q1*')].index, inplace=True)
data.drop(data.loc[(data['Vuosineljännes']== '2023Q2') | (data['Vuosineljännes']== '2023Q1')].index, inplace=True)

# drop before 2015 from data_prices
drop_idx_1 = data_pr.loc[(data_pr['Huoneluku'] == 'Yksiöt') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
drop_idx_2 = data_pr.loc[(data_pr['Huoneluku'] == 'Kaksiot') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
drop_idx_3 = data_pr.loc[(data_pr['Huoneluku'] == 'Kolmiot+') & (data_pr['Vuosineljännes']=='2015Q1')].index[0]
data_prd = pd.concat([data_pr.loc[data_pr['Huoneluku']=='Yksiöt'][data_pr.loc[data_pr['Huoneluku']=='Yksiöt'].index > drop_idx_1], 
                     data_pr.loc[data_pr['Huoneluku']=='Kaksiot'][data_pr.loc[data_pr['Huoneluku']=='Kaksiot'].index > drop_idx_2],
                     data_pr.loc[data_pr['Huoneluku']=='Kolmiot+'][data_pr.loc[data_pr['Huoneluku']=='Kolmiot+'].index > drop_idx_3]
                     ])

# print(data_prd.shape, data.shape)

data_merged = pd.merge(data_prd, data, on=['Vuosineljännes', 'Posti', 'Huoneluku', 'Kaupunki'], how='inner')
print(data_merged.head(3))

  Vuosineljännes Huoneluku Neliöhinta (EUR/m2) Lukumäärä_x  Posti  \
0         2015Q1    Yksiöt              6535.0          10  00120   
1         2015Q1    Yksiöt                 NaN           3  00130   
2         2015Q1    Yksiöt              6785.0          16  00140   

           Kaupunginosa_x  Kaupunki Lukumäärä_y Neliövuokra (eur/m2)  \
0     Punavuori-Bulevardi  Helsinki          95                21.52   
1         Kaartinkaupunki  Helsinki          24                27.01   
2  Kaivopuisto-Ullanlinna  Helsinki         141                23.26   

           Kaupunginosa_y  
0               Punavuori  
1         Kaartinkaupunki  
2  Kaivopuisto-Ullanlinna  


In [13]:
# Change Talotyyppi -> Huoneluku, Kauppojen lukumäärä, vara... -> Lukumäärä
data_prices.rename(columns={'Talotyyppi': 'Huoneluku', 'Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen' : 'Lukumäärä'}, inplace=True)

# values in Talotyyppi to same as the rent
to_rplc = {'Kerrostalo yksiöt': 'Yksiöt', 'Kerrostalo kaksiot': 'Kaksiot', 'Kerrostalo kolmiot+': 'Kolmiot+'}
data_prices['Huoneluku'].replace(to_rplc, inplace=True) 


data_prices
# WE NEED TO CHECK if they have put some inconsistent data somewhere like 2023Q2* =/= 2023Q2

,Vuosineljännes,Huoneluku,Neliöhinta (EUR/m2),Lukumäärä,Posti,Kaupunginosa,Kaupunki
0,2009Q1,Yksiöt,4751.0,36,00100,Helsinkikeskusta-Etu-Töölö,Helsinki
1,2009Q1,Yksiöt,4561.0,8,00120,Punavuori-Bulevardi,Helsinki
2,2009Q1,Yksiöt,NaN,1,00130,Kaartinkaupunki,Helsinki
3,2009Q1,Yksiöt,5128.0,12,00140,Kaivopuisto-Ullanlinna,Helsinki
4,2009Q1,Yksiöt,4472.0,40,00150,Punavuori-Eira-Hernesaari,Helsinki
...,...,...,...,...,...,...,...
98251,2023Q2*,Kolmiot+,NaN,2,33850,Multisilta,Tampere
98252,2023Q2*,Kolmiot+,3615.0,7,33870,Vuores,Tampere
98254,2023Q2*,Kolmiot+,3883.0,8,33900,Härmälä-Rantaperkiö,Tampere
98261,2023Q2*,Kolmiot+,NaN,.,34240,Kämmenteisko,Tampere


In [11]:
#FILL HOUSE PRICE NANS WITH NEW VALS BASED ON INDEX

#FILL NAN WITH INDEX (WORKING)

import numpy as np
import pandas as pd

#infindex=pd.read_csv('001_11xb_2023m08_20231004-151722.csv', sep=',', encoding='iso8859_4')

huoneet=['Yksiöt', 'Kaksiot', 'Kolmiot+']
setit=set()
uutuus_pr = pd.DataFrame().reindex_like(data_pr)
#print(uutuus.head())
i =0
for index, row in data_pr.iterrows():
    post=row['Posti']
    #print(post)
    if post not in setit:
        setit.add(post)
        for huone in huoneet:
            pala=data_pr.loc[(data_pr['Huoneluku']==huone) & (data_pr['Posti']==post)]
            print(pala.head(5))
            nanit=[]
            vall=0
            for ind, quarter in pala.iterrows():
                #print("quarter", quarter)
                temp=quarter['Neliöhinta (EUR/m2)']
                #print(pd.isna(temp))
                print(temp)
                if np.isnan(quarter['Neliöhinta (EUR/m2)']):
                    #print("added")
                    nanit.append(quarter['Vuosineljännes'])
                else:
                    vall=quarter['Neliöhinta (EUR/m2)']
                    quart=quarter['Vuosineljännes']
                    print(quart)
                    #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==quart)].values[0]
                    #inde=inde['Index']/100
                    inde = house_inds.loc[house_inds['Vuosineljännes'] == quart, 'Index'].values[0] / 100
                    vall=round(vall/inde, 2)
                    for nani in nanit:
                        #print("nani ", nani)
                        #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==nani)].values[0]
                        #inde=inde['Index']/100
                        inde = house_inds.loc[house_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == nani, 'Neliöhinta (EUR/m2)'] = round(vall * inde, 2)
                        print(pala.loc[pala['Vuosineljännes'] == nani, 'Neliöhinta (EUR/m2)'])
                    nanit=[]
            #print("through 1")
            if vall!=0:
                if len(nanit)>0:
                    for nani in nanit:
                        #print("ff", nani)
                        #inde=rent_inds.loc[(rent_inds['Vuosineljännes']==nani)].values[0]
                        #inde=inde['Index']/100
                        inde = house_inds.loc[house_inds['Vuosineljännes'] == nani, 'Index'].values[0] / 100
                        pala.loc[pala['Vuosineljännes'] == str(nani), 'Neliöhinta (EUR/m2)'] = round(vall * inde, 2)
                        #print(pala.loc[pala['Vuosineljännes'] == str(nani), 'Neliövuokra (eur/m2)'], round(vall * inde, 2))
            nanit=[]
                #pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
            #print(pala.head(5))
            #print("through")
            for ind, row in pala.iterrows():
                uutuus_pr.loc[i]=row
                i+=1
uutuus_pr=uutuus_pr.dropna(subset=['Kaupunki'])  

      Vuosineljännes Huoneluku Neliöhinta (EUR/m2) Lukumäärä  Posti  \
41065         2015Q1    Yksiöt              6535.0        10  00120   
42776         2015Q2    Yksiöt              6651.0        11  00120   
44487         2015Q3    Yksiöt              7401.0        11  00120   
46198         2015Q4    Yksiöt              6387.0        10  00120   
47909         2016Q1    Yksiöt              6447.0        12  00120   

              Kaupunginosa  Kaupunki  
41065  Punavuori-Bulevardi  Helsinki  
42776  Punavuori-Bulevardi  Helsinki  
44487  Punavuori-Bulevardi  Helsinki  
46198  Punavuori-Bulevardi  Helsinki  
47909  Punavuori-Bulevardi  Helsinki  
6535.0
2015Q1
6651.0
2015Q2
7401.0
2015Q3
6387.0
2015Q4
6447.0
2016Q1
7284.0
2016Q2
6485.0
2016Q3
6769.0
2016Q4
nan
7604.0
2017Q2
54753    7652.53
Name: Neliöhinta (EUR/m2), dtype: object
8131.0
2017Q3
7551.0
2017Q4
7899.0
2018Q1
7534.0
2018Q2
7269.0
2018Q3
8145.0
2018Q4
8575.0
2019Q1
8591.0
2019Q2
8115.0
2019Q3
7667.0
2019Q4
8407.0
2020

In [21]:
data_merged = pd.merge(uutuus_pr, uutuus, on=['Vuosineljännes', 'Posti', 'Huoneluku', 'Kaupunki'], how='inner')
data_merged=data_merged.dropna()
data_merged['difference'] = round(data_merged['Neliöhinta (EUR/m2)'] / data_merged['Neliövuokra (eur/m2)'], 2)
data_merged.to_csv('merged_data.csv')

In [28]:
#GET THE DATA FOR YEAR 2023
curr_data=data_merged.loc[(data_merged['Vuosineljännes']=="2022Q4")]
curr_data_single=curr_data.loc[(data_merged['Huoneluku']=="Yksiöt")]
curr_data.to_csv('2022_quarte4.csv')
curr_data_single.to_csv('2022_quarte4_singles.csv')